In [0]:
import tensorflow as tf

import numpy as np
import os
import time
import re

In [0]:
path_to_file = '/content/drive/My Drive/Colab Notebooks/shakespeare.txt'

In [0]:
# read data file
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [0]:
def default_dic_words (corpus_list):
    counter = dict()
    for kw in corpus_list:
        counter[kw] = counter.setdefault(kw, 0) + 1
        
    return counter

In [0]:
def count_distinct_words (corpus_list):
    return len(default_dic_words(corpus_list))

In [0]:
# convert to lower
# remove all new line characters
# do not remove any other characters 
# because those characters would become a very important member of a formal sentence
# remove extra space
# remove empty entries
# return a list of tokens
# The reason that I don't want to delete - and ' is 
# that they are special characters that would make unique words at Shakespear's time
def preprocessing_txt(txt_data):
    # remove new line chars
    temp_txt = re.sub(r'\n+', r' ', txt_data)
    # remove back slash
    temp_txt = re.sub(r'\\', r'', temp_txt)
    temp_txt = re.sub(r' +', r' ', temp_txt)
    temp_txt = temp_txt.lower()
    temp_txt = temp_txt.split(' ')
    return_txt_list = []
    for i in temp_txt:
        if i.strip() != '':
            return_txt_list.append(i.strip())
    return return_txt_list

In [0]:
new_txt_data = preprocessing_txt(text)

In [0]:
# apply the function
vocab = sorted(set(preprocessing_txt(text)))

In [9]:
vocab

['!',
 "!'",
 "!'by",
 "!'t",
 "!'twas",
 '!,',
 '!a',
 '!about',
 '!alive',
 '!all',
 '!and',
 '!any',
 '!are',
 '!as',
 '!ay',
 '!burn',
 '!come',
 '!cuckold',
 '!did',
 '!do',
 '!donc',
 '!down',
 '!even',
 '!follow',
 '!for',
 '!give',
 "!give't",
 '!handkerchief',
 '!have',
 '!he',
 '!hear',
 '!help',
 '!his',
 '!hoo',
 '!how',
 '!i',
 '!if',
 '!it',
 '!jack',
 '!kill',
 '!let',
 '!m',
 '!matter',
 '!my',
 '!nay',
 '!o',
 '!on',
 '!out',
 '!please',
 '!poor',
 '!remain',
 "!reveal'd",
 '!seek',
 '!sir',
 '!sixpence',
 '!slay',
 '!speak',
 '!speak-a',
 '!stay',
 '!taught',
 '!that',
 '!the',
 '!their',
 '!they',
 '!this',
 '!to',
 '!videlicet',
 '!vouchsafe',
 '!was',
 '!wast',
 '!we',
 '!weapons',
 '!well',
 '!what',
 '!when',
 '!who',
 "!who's",
 '!whose',
 '!why',
 '!won',
 '!would',
 '!you',
 '"doubt',
 '"to',
 '"upon',
 '&c',
 "''tis",
 "''twas",
 "'a",
 "'above",
 "'accost",
 "'accost'",
 "'accurs'd",
 "'achilles",
 "'adieu",
 "'after",
 "'against",
 "'aged",
 "'agrippa",
 "'

In [0]:
# 创建从非重复字符到索引的映射
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in new_txt_data])

In [11]:
text_as_int

array([ 1012, 16111,  8348, ..., 13861, 21602,   823])

In [12]:
# input length
seq_length = 100
examples_per_epoch = len(text)//seq_length

# create training datasets
vocab_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in vocab_dataset.take(5):
  print(idx2char[i.numpy()])

a
midsummer-night's
dream
now
,


In [13]:
sentences = vocab_dataset.batch(seq_length+1, drop_remainder=True)

for item in sentences.take(5):
  print(repr(' '.join(idx2char[item.numpy()])))

"a midsummer-night's dream now , fair hippolyta , our nuptial hour draws on apace : four happy days bring in another moon ; but o ! methinks how slow this old moon wanes ; she lingers my desires , like to a step dame , or a dowager long withering out a young man's revenue . four days will quickly steep themselves in night ; four nights will quickly dream away the time ; and then the moon , like to a silver bow new-bent in heaven , shall behold the night of our solemnities . go , philostrate , stir"
"up the athenian youth to merriments ; awake the pert and nimble spirit of mirth ; turn melancholy forth to funerals ; the pale companion is not for our pomp . hippolyta , i woo'd thee with my sword , and won thy love doing thee injuries ; but i will wed thee in another key , with pomp , with triumph , and with revelling . happy be theseus , our renowned duke ! thanks , good egeus : what's the news with thee ? full of vexation come i , with complaint against my child , my daughter hermia"
".

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sentences.map(split_input_target)

In [15]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(' '.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(' '.join(idx2char[target_example.numpy()])))

Input data:  "a midsummer-night's dream now , fair hippolyta , our nuptial hour draws on apace : four happy days bring in another moon ; but o ! methinks how slow this old moon wanes ; she lingers my desires , like to a step dame , or a dowager long withering out a young man's revenue . four days will quickly steep themselves in night ; four nights will quickly dream away the time ; and then the moon , like to a silver bow new-bent in heaven , shall behold the night of our solemnities . go , philostrate ,"
Target data: "midsummer-night's dream now , fair hippolyta , our nuptial hour draws on apace : four happy days bring in another moon ; but o ! methinks how slow this old moon wanes ; she lingers my desires , like to a step dame , or a dowager long withering out a young man's revenue . four days will quickly steep themselves in night ; four nights will quickly dream away the time ; and then the moon , like to a silver bow new-bent in heaven , shall behold the night of our solemnities 

In [16]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1012 ('a')
  expected output: 16111 ("midsummer-night's")
Step    1
  input: 16111 ("midsummer-night's")
  expected output: 8348 ('dream')
Step    2
  input: 8348 ('dream')
  expected output: 17312 ('now')
Step    3
  input: 17312 ('now')
  expected output: 610 (',')
Step    4
  input: 610 (',')
  expected output: 9646 ('fair')


In [17]:
# batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:

vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
# the function actually uses softmax as the output
# from_logits indicates whether using softmax or not
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:

checkpoint_dir = './training_checkpoints'


checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=30

In [29]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
151/151 [==============================] - 87s 578ms/step - loss: 5.7699
Epoch 2/30
151/151 [==============================] - 87s 577ms/step - loss: 5.7374
Epoch 3/30
151/151 [==============================] - 87s 577ms/step - loss: 5.7053
Epoch 4/30
151/151 [==============================] - 87s 574ms/step - loss: 5.6725
Epoch 5/30
151/151 [==============================] - 87s 575ms/step - loss: 5.6477
Epoch 6/30
151/151 [==============================] - 87s 574ms/step - loss: 5.6216
Epoch 7/30
151/151 [==============================] - 87s 575ms/step - loss: 5.6011
Epoch 8/30
151/151 [==============================] - 87s 575ms/step - loss: 5.5820
Epoch 9/30
151/151 [==============================] - 87s 577ms/step - loss: 5.5623
Epoch 10/30
151/151 [==============================] - 87s 578ms/step - loss: 5.5456
Epoch 11/30
151/151 [==============================] - 87s 577ms/step - loss: 5.5277
Epoch 12/30
151/151 [==============================] - 87s 577ms/step - lo

In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [0]:
model_load = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model_load.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model_load.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):
  num_generate = 15

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []
  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
      predictions = model_load(input_eval)
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return ' '.join(text_generated)

In [33]:
print('the '+generate_text(model, start_string=['the']))

the razed months good feeble ! 'an like a battle : but there's framing near .
